# Install

# Import

In [ ]:
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)
import re,os,json
import logging
logging.getLogger('pdfminer.pdfinterp').setLevel(logging.ERROR)
logging.getLogger('pdfminer.layout').setLevel(logging.ERROR)
from modules.schedule import ITR1Sections

# fn

In [ ]:
from modules.process_pdf import process_pdf
from modules.schedule import ITR1Sections

# Code

In [ ]:
# input_file_path = 'OUTPUT/ ITR 1 AY 2023-24.pdf'
# output_file_path = input_file_path.replace(".pdf", "_extracted.txt")
# config_path = "config/ITR1_header.json"
# itr1 = ITR1Sections(input_file_path, output_file_path, config_path)
# itr1

In [ ]:
# itr1.debug["raw_extracted"]
# itr1.debug["section_ranges"]
# itr1.debug["metadata"]
# itr1.debug["patterns"]
# itr1.debug["final_dataframes"]["TDS"]

In [ ]:
# meta=itr1.debug["section_ranges"]['SCHEDULE_80G_C']
# s, e = meta["start"], meta["end"]
# df_raw = pd.DataFrame(itr1.debug["raw_extracted"][s:e])
# df_raw

In [ ]:
# assert False

In [ ]:
pdf_directory = "OUTPUT"
config_path = "config/ITR1_header.json"

all_pdfs = [f for f in os.listdir(pdf_directory) if f.lower().endswith(".pdf")]
print(f"Found {len(all_pdfs)} PDFs: {all_pdfs}")

all_itr1_sections = {}
for pdf_filename in all_pdfs:
    input_file = os.path.join(pdf_directory, pdf_filename)
    output_file = input_file.replace(".pdf", "_extracted.txt")
    print(f"Processing {pdf_filename}... ",end="")
    try:
        itr1 = ITR1Sections(input_file, output_file, config_path)
        print(itr1.pan)
        key = itr1.ack or pdf_filename
        all_itr1_sections[key] = itr1
    except Exception as e:
        print(f"Error processing {pdf_filename}: {e}")

print("\nProcessing complete.")
print(f"Successfully processed {len(all_itr1_sections)} of {len(all_pdfs)} files.")

In [ ]:
metadata_df = pd.DataFrame([
    {"Acknowledgement": k, "pan_number": v.pan, "dof_date": v.dof, "itr1_object": v.dataframes}
    for k, v in all_itr1_sections.items()
])
metadata_df["dof_date"] = pd.to_datetime(metadata_df["dof_date"], errors="coerce", dayfirst=True)
with open(config_path, encoding="utf-8") as f:
    config = json.load(f)
for pan, group in metadata_df.groupby("pan_number"):
    group_sorted = group.sort_values("dof_date")
    available_sections = set().union(*[obj.keys() for obj in group_sorted["itr1_object"]])
    output_file = os.path.join(pdf_directory, f"{pan}.xlsx")

    with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
        for section in config.keys():
            if section in available_sections:
                dfs = []
                for _, row in group_sorted.iterrows():
                    if section in row["itr1_object"]:
                        df = row["itr1_object"][section].copy()
                        # add empty row at end
                        df.loc[len(df)] = [pd.NA] * len(df.columns)
                        dfs.append(df)

                if dfs:
                    final_df = pd.concat(dfs, ignore_index=True)
                    final_df.to_excel(writer, sheet_name=section, index=False)

                    # Auto-adjust column widths
                    worksheet = writer.sheets[section]
                    for idx, col in enumerate(final_df.columns):
                        # Convert all values to string and get max length
                        max_len = max(
                            [len(str(col))] + [len(str(val)) for val in final_df[col].values]
                        )
                        worksheet.set_column(idx, idx, max_len + 2)  # +2 for padding

    print(f"✅ Exported {output_file}")

In [ ]:
assert False

In [ ]:
# Build metadata DataFrame directly
metadata_df = pd.DataFrame([
    {"Acknowledgement": k, "pan_number": v.pan, "dof_date": v.dof, "itr1_object": v.dataframes}
    for k, v in all_itr1_sections.items()
])
# metadata_df['dof_date'] = pd.to_datetime(metadata_df['dof_date'], format='%d-%b-%Y', errors='coerce')
# metadata_df = metadata_df.sort_values(by=['pan_number', 'dof_date'], ascending=[True, True])
metadata_df

In [ ]:
for pan_number in metadata_df["pan_number"].unique():
    itr1_list=metadata_df[metadata_df["pan_number"]==pan_number][["itr1_object"]]
    for itr1 in itr1_list:
        print(itr1.dataframes.keys())
        break
    break

In [ ]:
itr1_list=metadata_df["itr1_object"].to_list()

In [ ]:
itr1=itr1_list[-1]
itr2=itr1_list[-2]

In [ ]:
df_EXEMPT_INCOME= itr1.get_section("EXEMPT_INCOME")
display(df_EXEMPT_INCOME)

In [ ]:
df_EXEMPT_INCOME= itr2.get_section("EXEMPT_INCOME")
display(df_EXEMPT_INCOME)

In [ ]:
assert False

In [ ]:
# itr1.export_to_excel(excel_file_path)

In [ ]:
# itr1.start_pattern

In [ ]:
# itr1.end_pattern

In [ ]:
# itr1.extracted_data[33:34]

In [ ]:
df_temp = pd.DataFrame(itr1.extracted_data[147:188])
# df_temp = clean_row(df_temp)
# df_temp = apply_dynamic_headers(df_temp, itr1.config,"ScheduleEA10_13A")
# df_temp

In [ ]:
# cols="SCHEDULE_80D"
# print(itr1.start_pattern.get(cols,"<No found>"))
# print(itr1.end_pattern.get(cols,"<No found>"))
# print(itr1.hdr_row_map.get(cols,"<No found>"))

In [ ]:
# extract_sections(itr1.extracted_data[147:188], itr1.start_pattern,itr1.end_pattern,itr1.hdr_row_map)

In [ ]:
cols="SCHEDULE_80D"
itr1.sections.get(cols,"<No found>")

In [ ]:
df_temp = itr1.get_section(cols)
# display(df_temp)

In [ ]:
# df_temp.to_excel("del.xlsx",index=False)

In [ ]:
# assert False

In [ ]:
itr1.acknowledgement,itr1.dof,itr1.pan_number

In [ ]:
df_EXEMPT_INCOME= itr1.get_section("EXEMPT_INCOME")
display(df_EXEMPT_INCOME)

In [ ]:
df_LTCG112A= itr1.get_section("LTCG112A")
display(df_LTCG112A)

In [ ]:
# df_PART_C= itr1.get_section("PART_C")
# display(df_PART_C)

In [ ]:
# df_PART_D= itr1.get_section("PART_D")
# display(df_PART_D)

In [ ]:
df_PART_E= itr1.get_section("PART_E")
display(df_PART_E)

In [ ]:
df_EA10_13A= itr1.get_section("ScheduleEA10_13A")
display(df_EA10_13A)

In [ ]:
df_80u = itr1.get_section("SECTION_80U")
display(df_80u)

In [ ]:
df_80dd = itr1.get_section("SECTION_80DD")
display(df_80dd)

In [ ]:
df_80G_A= itr1.get_section("SCHEDULE_80G_A")
display(df_80G_A)

In [ ]:
df_80G_B= itr1.get_section("SCHEDULE_80G_B")
display(df_80G_B)

In [ ]:
df_80G_C= itr1.get_section("SCHEDULE_80G_C")
display(df_80G_C)

In [ ]:
df_80G_D= itr1.get_section("SCHEDULE_80G_D")
display(df_80G_D)

In [ ]:
df_80GGA= itr1.get_section("SCHEDULE_80GGA")
display(df_80GGA)

In [ ]:
df_80GGC= itr1.get_section("SCHEDULE_80GGC")
display(df_80GGC)

In [ ]:
df_it= itr1.get_section("SCHEDULE_IT")
display(df_it)

In [ ]:
df_TDS1= itr1.get_section("SCHEDULE_TDS1")
display(df_TDS1)

In [ ]:
df_TDS2= itr1.get_section("SCHEDULE_TDS2")
display(df_TDS2)

In [ ]:
df_TDS3= itr1.get_section("SCHEDULE_TDS3")
display(df_TDS3)

In [ ]:
df_TCS= itr1.get_section("SCHEDULE_TCS")
display(df_TCS)